# Día 3: Manejo de Errores y Excepciones

## Descripción General

El manejo de errores es una parte fundamental de la programación profesional. En lugar de permitir que tu programa se detenga abruptamente cuando algo sale mal, Python te proporciona un sistema robusto de excepciones que te permite anticipar, capturar y responder a errores de manera elegante.

En este notebook aprenderás a usar `try/except` para capturar errores, crear excepciones personalizadas que comuniquen problemas específicos de tu dominio, y propagar errores de manera apropiada a través de tu código. Estas habilidades son esenciales para construir aplicaciones confiables y fáciles de depurar.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Usar bloques `try/except/else/finally` para manejar excepciones de manera efectiva
2. Capturar excepciones específicas y múltiples tipos de excepciones
3. Crear excepciones personalizadas que representen errores específicos de tu dominio
4. Propagar excepciones apropiadamente usando `raise` y re-lanzamiento
5. Aplicar mejores prácticas para el manejo de errores en código de producción

## 1. Fundamentos de Try/Except

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Tu pipeline de ML procesa 10,000 archivos CSV. En el archivo 7,342 hay un valor corrupto. Sin manejo de errores: **todo el pipeline crashea** 💥 Pierdes 3 horas de procesamiento. Tienes que reiniciar desde cero.

Con manejo de errores: Logeas el archivo problemático, continúas con los otros 2,658 archivos. **Procesamiento completo en 4 horas** ✅ Arreglas el archivo corrupto después.

**Ejemplo concreto para juniors**:

API que calcula estadísticas. Usuario envía lista vacía. Sin try/except: `ZeroDivisionError`, API retorna 500, usuario ve "Internal Server Error". Con try/except: API retorna 400 con mensaje claro "Cannot calculate average of empty list".

**Consecuencias de NO usarlo**:
- **Crashes en producción** → usuarios ven errores crípticos
- **Pérdida de trabajo** → procesamiento largo se pierde
- **Debugging difícil** → no sabes qué causó el error
- **Mala experiencia** → usuarios frustrados
- **Datos perdidos** → transacciones a medias

### 📚 El Concepto

**Definición técnica**:

`try/except` permite **capturar y manejar excepciones** sin que el programa crashee. Python ejecuta código en `try`, si falla, salta a `except` correspondiente.

**Cómo funciona internamente**:
1. Python ejecuta código en bloque `try`
2. Si ocurre excepción → busca bloque `except` que coincida
3. Si encuentra match → ejecuta ese `except`
4. Si no encuentra match → propaga excepción hacia arriba
5. Después de `except` → continúa ejecución normal

**Terminología clave**:
- **Exception**: Objeto que representa un error
- **Raise**: Lanzar una excepción
- **Catch**: Capturar una excepción
- **Propagate**: Pasar excepción al caller
- **Stack trace**: Historial de llamadas hasta el error

**Jerarquía de excepciones**:
```
BaseException
├── Exception (captura la mayoría)
│   ├── ValueError
│   ├── TypeError
│   ├── KeyError
│   ├── IndexError
│   └── ZeroDivisionError
├── KeyboardInterrupt (NO capturar)
└── SystemExit (NO capturar)
```

### Ejemplo: Código sin Manejo de Errores (MAL)

In [ ]:
# BAD: No error handling - program crashes
def calculate_average_bad(numbers):
    """
    Calculate average without error handling.
    
    :param numbers: List of numbers
    :type numbers: list
    :return: Average value
    :rtype: float
    """
    return sum(numbers) / len(numbers)

# This will crash if list is empty
# result = calculate_average_bad([])  # ZeroDivisionError!

### Ejemplo: Código con Manejo de Errores (BIEN)

In [ ]:
# GOOD: Proper error handling
def calculate_average_good(numbers):
    """
    Calculate average with error handling.
    
    :param numbers: List of numbers
    :type numbers: list
    :return: Average value or None if list is empty
    :rtype: float | None
    """
    try:
        return sum(numbers) / len(numbers)
    except ZeroDivisionError:
        print("Error: Cannot calculate average of empty list")
        return None
    except TypeError:
        print("Error: List contains non-numeric values")
        return None

# Now it handles errors gracefully
print(calculate_average_good([]))  # None
print(calculate_average_good([1, 2, 3, 4, 5]))  # 3.0

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **Captura específica** → `except ValueError` no `except:` (evita ocultar bugs)
2. **Jerarquía importa** → `except Exception` captura casi todo (cuidado)
3. **Múltiples except** → orden de más específico a más genérico
4. **No captures KeyboardInterrupt** → usuario debe poder detener programa
5. **Logging antes de continuar** → registra el error para debugging

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Qué errores específicos pueden ocurrir aquí?"
  - Sabes cuáles → captura esos específicamente ✅
  - No sabes → deja que falle y aprende ❌ (no uses `except:` genérico)

- **Pregúntate**: "¿Puedo recuperarme de este error?"
  - SÍ → captura y maneja ✅
  - NO → deja que se propague ✅

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Sabes qué errores específicos pueden ocurrir
  - Puedes recuperarte o dar mensaje útil
  - Necesitas liberar recursos (archivos, conexiones)
- ❌ **NO usar cuando**:
  - No sabes qué error esperar (no uses `except:` genérico)
  - El error indica bug en tu código (déjalo fallar)
  - Solo quieres "silenciar" errores (antipatrón)

**Referencia oficial:** [Python Tutorial - Errors and Exceptions](https://docs.python.org/tutorial/errors.html)

## 2. Bloques Else y Finally

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Tu script procesa 1,000 archivos CSV. Abre archivo, procesa datos, cierra archivo. En el archivo 347 hay error de parsing. **Sin `finally`: archivo queda abierto** 💥 Después de 653 archivos más, tienes 654 file handles abiertos. Sistema operativo dice "too many open files". Todo crashea.

Con `finally`: Archivo SIEMPRE se cierra, haya error o no. **Procesamiento completo sin memory leaks** ✅

**Ejemplo concreto para juniors**:

Conectas a base de datos, ejecutas query, cierras conexión. Query falla. Sin `finally`: conexión queda abierta. Después de 100 requests, pool de conexiones agotado. Nuevos usuarios no pueden conectar.

**Consecuencias de NO usarlo**:
- **Resource leaks** → archivos/conexiones quedan abiertos
- **Memory leaks** → memoria no se libera
- **Límites del OS** → "too many open files" después de N operaciones
- **Degradación progresiva** → sistema se vuelve más lento con el tiempo
- **Crashes inesperados** → falla después de horas funcionando

### 📚 El Concepto

**Definición técnica**:

`else` ejecuta solo si NO hubo excepción. `finally` ejecuta SIEMPRE, haya o no excepción. Útil para limpieza de recursos.

**Cómo funciona internamente**:
1. Python ejecuta bloque `try`
2. Si hay excepción → salta a `except` correspondiente
3. Si NO hay excepción → ejecuta bloque `else` (si existe)
4. **SIEMPRE** ejecuta bloque `finally` (si existe)
5. Si hay excepción no capturada → `finally` se ejecuta antes de propagar

**Terminología clave**:
- **else**: Código que solo corre si NO hubo excepción
- **finally**: Código que SIEMPRE corre (limpieza)
- **Resource cleanup**: Liberar recursos (archivos, conexiones, locks)
- **Guaranteed execution**: Código que debe ejecutarse sí o sí

### Estructura Completa de Try/Except

In [ ]:
def read_file_safely(filename):
    """
    Read file with complete error handling.
    
    :param filename: Path to file
    :type filename: str
    :return: File contents or None
    :rtype: str | None
    """
    file_handle = None
    try:
        file_handle = open(filename, 'r')
        content = file_handle.read()
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found")
        return None
    except PermissionError:
        print(f"Error: No permission to read '{filename}'")
        return None
    else:
        # Only executes if NO exception occurred
        print(f"Successfully read {len(content)} characters")
        return content
    finally:
        # ALWAYS executes, even if exception occurred
        if file_handle:
            file_handle.close()
            print("File closed")

# Test with non-existent file
result = read_file_safely("nonexistent.txt")

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **`else` vs código después** → `else` solo corre si NO hubo excepción (más claro)
2. **`finally` SIEMPRE corre** → incluso si hay `return` en `try` o `except`
3. **Limpieza en `finally`** → cerrar archivos, conexiones, locks
4. **`finally` corre antes de propagar** → si excepción no capturada, `finally` corre primero
5. **Mejor usar context managers** → `with` es más limpio que `try/finally` para recursos

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Este código debe correr solo si todo salió bien?"
  - SÍ → usa `else` ✅
  - NO → ponlo después del `try/except` ✅

- **Pregúntate**: "¿Este código debe correr SÍ O SÍ?"
  - SÍ → usa `finally` ✅
  - NO → ponlo en `else` o después ✅

- **Pregúntate**: "¿Estoy manejando un recurso (archivo, conexión)?"
  - SÍ → mejor usa `with` (context manager) ✅
  - NO → `try/finally` está bien ✅

**Cuándo usar / NO usar**:
- ✅ **Usar `else` cuando**:
  - Código solo debe correr si NO hubo excepción
  - Quieres separar "happy path" de manejo de errores
  - Mejora legibilidad vs poner código después
- ✅ **Usar `finally` cuando**:
  - Necesitas liberar recursos (cerrar archivos, conexiones)
  - Código debe correr incluso si hay excepción
  - Limpieza que no puede faltar
- ❌ **NO usar cuando**:
  - Puedes usar `with` (context manager) → más limpio
  - No hay recursos que limpiar → innecesario

**Referencia oficial:** [Built-in Exceptions](https://docs.python.org/3/library/exceptions.html)

### Pregunta de Comprensión

¿Cuál es la diferencia entre poner código después del bloque `try/except` versus ponerlo en el bloque `else`?

## 3. Excepciones Personalizadas

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Tu API de ML recibe requests. Validación falla y lanzas `ValueError("Invalid input")`. Cliente ve error genérico. **No sabe QUÉ está mal** 💥 Prueba 10 combinaciones diferentes. Frustrante.

Con excepción personalizada `InvalidModelInputError(field="age", value=-5, reason="must be positive")`: Cliente ve exactamente qué arreglar. **Fix en 30 segundos** ✅

**Ejemplo concreto para juniors**:

Sistema de pagos. Saldo insuficiente. Lanzas `ValueError("Error")`. Usuario ve "Error". ¿Qué error? ¿Tarjeta inválida? ¿Saldo bajo? ¿Servicio caído?

Con `InsufficientFundsError(balance=50, amount=100, shortfall=50)`: Usuario ve "Saldo insuficiente: tienes $50, necesitas $100, faltan $50". **Mensaje claro y accionable** ✅

**Consecuencias de NO usarlo**:
- **Debugging difícil** → `ValueError` genérico no dice qué falló
- **Mala UX** → usuarios no saben qué arreglar
- **Logs inútiles** → "ValueError: Invalid" aparece 1000 veces, ¿cuál es cuál?
- **No puedes capturar específicamente** → `except ValueError` captura TODO
- **Pérdida de contexto** → no puedes incluir datos relevantes

### 📚 El Concepto

**Definición técnica**:

Excepciones personalizadas son clases que heredan de `Exception`. Representan errores específicos de tu dominio. Pueden incluir atributos con contexto relevante.

**Cómo funciona internamente**:
1. Defines clase que hereda de `Exception` (o subclase)
2. `__init__` acepta parámetros relevantes al error
3. Guardas parámetros como atributos (`self.balance`, `self.amount`)
4. Llamas `super().__init__(mensaje)` para mensaje de error
5. Código que captura puede acceder a atributos para más contexto

**Terminología clave**:
- **Custom Exception**: Excepción específica de tu dominio
- **Exception Hierarchy**: Árbol de excepciones (más específicas heredan de más generales)
- **Error Context**: Información adicional sobre el error
- **Domain-Specific**: Representa conceptos de tu aplicación

**Convenciones de nombres**:
- Terminar en `Error`: `InvalidEmailError`, `InsufficientFundsError`
- Descriptivo: `DatabaseConnectionError` no `DBError`
- Específico: `UserNotFoundError` no `NotFoundError`

### Ejemplo: Excepciones Genéricas (MAL)

In [ ]:
# BAD: Using generic exceptions
def withdraw_money_bad(balance, amount):
    """
    Withdraw money using generic exceptions.
    
    :param balance: Current balance
    :type balance: float
    :param amount: Amount to withdraw
    :type amount: float
    :return: New balance
    :rtype: float
    """
    if amount > balance:
        raise ValueError("Not enough money")  # Too generic!
    return balance - amount

### Ejemplo: Excepciones Personalizadas (BIEN)

In [ ]:
# GOOD: Custom exceptions with clear meaning
class InsufficientFundsError(Exception):
    """
    Raised when attempting to withdraw more money than available.
    
    :param balance: Current balance
    :type balance: float
    :param amount: Attempted withdrawal amount
    :type amount: float
    """
    def __init__(self, balance, amount):
        self.balance = balance
        self.amount = amount
        self.shortfall = amount - balance
        super().__init__(
            f"Insufficient funds: balance={balance}, "
            f"requested={amount}, shortfall={self.shortfall}"
        )

class NegativeAmountError(Exception):
    """
    Raised when attempting to withdraw negative amount.
    """
    pass

def withdraw_money_good(balance, amount):
    """
    Withdraw money with custom exceptions.
    
    :param balance: Current balance
    :type balance: float
    :param amount: Amount to withdraw
    :type amount: float
    :return: New balance
    :rtype: float
    :raises InsufficientFundsError: If balance is insufficient
    :raises NegativeAmountError: If amount is negative
    """
    if amount < 0:
        raise NegativeAmountError("Cannot withdraw negative amount")
    if amount > balance:
        raise InsufficientFundsError(balance, amount)
    return balance - amount

# Now we can catch specific errors
try:
    new_balance = withdraw_money_good(100.0, 150.0)
except InsufficientFundsError as e:
    print(f"Transaction failed: {e}")
    print(f"You need ${e.shortfall:.2f} more")

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **Hereda de `Exception`** → no de `BaseException` (reservado para sistema)
2. **Nombres terminan en `Error`** → `InvalidEmailError` no `InvalidEmail`
3. **Incluye contexto** → atributos con datos relevantes (`balance`, `amount`)
4. **Mensaje descriptivo** → explica QUÉ falló y POR QUÉ
5. **Jerarquía si necesario** → `PaymentError` → `InsufficientFundsError`, `InvalidCardError`

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Este error es específico de mi dominio?"
  - SÍ → crea excepción personalizada ✅
  - NO → usa built-in (`ValueError`, `TypeError`) ✅

- **Pregúntate**: "¿Necesito capturar este error específicamente?"
  - SÍ → excepción personalizada permite `except InsufficientFundsError` ✅
  - NO → built-in está bien ✅

- **Pregúntate**: "¿Hay contexto útil que incluir?"
  - SÍ → añade atributos (`self.balance`, `self.shortfall`) ✅
  - NO → excepción simple está bien ✅

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Error es específico de tu dominio (negocio, ML, etc.)
  - Necesitas capturar este error específicamente
  - Quieres incluir contexto adicional (atributos)
  - Mensaje de error necesita ser muy específico
  - Múltiples lugares lanzan el mismo tipo de error
- ❌ **NO usar cuando**:
  - Error genérico (validación simple) → `ValueError` suficiente
  - Solo se lanza en un lugar → overhead innecesario
  - No necesitas capturarlo específicamente

**Referencia oficial:** [PEP 352 - Required Superclass for Exceptions](https://www.python.org/dev/peps/pep-0352/)

## 4. Propagación de Excepciones

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Tu función `load_model()` llama a `open_file()` que lanza `FileNotFoundError`. Capturas el error, logeas, y... ¿ahora qué? Si no re-lanzas, el caller piensa que todo salió bien. **Continúa con modelo `None`, crashea 50 líneas después** 💥 Stack trace no muestra el error original.

Con re-lanzamiento: Logeas el error Y lo propagas. Caller puede decidir qué hacer. **Stack trace completo, debugging fácil** ✅

**Ejemplo concreto para juniors**:

API REST. Función de bajo nivel lanza `psycopg2.DatabaseError`. Usuario ve "psycopg2.DatabaseError: connection failed". **¿Qué es psycopg2?** 🤔 Confuso.

Transformas a `ServiceUnavailableError("Database temporarily unavailable")`. Usuario ve mensaje claro. **Logs internos tienen ambos errores** ✅

**Consecuencias de NO usarlo**:
- **Errores silenciados** → capturas pero no propagas, caller no sabe que falló
- **Stack traces incompletos** → no se ve dónde originó el error
- **Debugging imposible** → error ocurre lejos de donde se capturó
- **Abstracciones rotas** → detalles de implementación (psycopg2) expuestos al usuario
- **Logs confusos** → no se ve la cadena de errores

### 📚 El Concepto

**Definición técnica**:

**Re-lanzar**: Capturar excepción, hacer algo (logging), y lanzarla de nuevo con `raise`.
**Encadenar**: Transformar excepción de bajo nivel en una de alto nivel, preservando la original con `from`.

**Cómo funciona internamente**:
1. **Re-lanzar**: `raise` sin argumentos re-lanza la excepción actual
2. **Encadenar**: `raise NewError() from original` crea nueva excepción
3. Python guarda original en `__cause__` de la nueva
4. Stack trace muestra ambas excepciones
5. Código que captura puede acceder a `__cause__` para contexto completo

**Terminología clave**:
- **Re-raise**: Lanzar de nuevo la misma excepción
- **Exception Chaining**: Encadenar excepciones con `from`
- **__cause__**: Atributo con la excepción original
- **Abstraction Layer**: Nivel de abstracción (bajo nivel vs alto nivel)

**Patrones comunes**:
```python
# Patrón 1: Log y re-lanza
try:
    risky_operation()
except ValueError as e:
    logger.error(f"Failed: {e}")
    raise  # Re-lanza la misma excepción

# Patrón 2: Transforma y encadena
try:
    low_level_operation()
except LowLevelError as e:
    raise HighLevelError("Operation failed") from e
```

### Ejemplo: Re-lanzar Excepciones

In [ ]:
def process_user_data(user_id):
    """
    Process user data with exception re-raising.
    
    :param user_id: User identifier
    :type user_id: int
    :raises ValueError: If user_id is invalid
    """
    try:
        # Simulate database operation
        if user_id < 0:
            raise ValueError("User ID must be positive")
        print(f"Processing user {user_id}")
    except ValueError as e:
        # Log the error
        print(f"[LOG] Error processing user {user_id}: {e}")
        # Re-raise the same exception
        raise  # No argument = re-raise the current exception

# The caller can also handle it
try:
    process_user_data(-5)
except ValueError:
    print("Caller: Caught the re-raised exception")

### Ejemplo: Encadenar Excepciones

In [ ]:
class DataProcessingError(Exception):
    """
    High-level exception for data processing errors.
    """
    pass

def parse_config_file(filename):
    """
    Parse configuration file with exception chaining.
    
    :param filename: Path to config file
    :type filename: str
    :return: Parsed configuration
    :rtype: dict
    :raises DataProcessingError: If parsing fails
    """
    try:
        with open(filename, 'r') as f:
            # Simulate parsing
            content = f.read()
            if not content:
                raise ValueError("Empty config file")
            return {"config": "data"}
    except (FileNotFoundError, ValueError) as e:
        # Transform low-level exception to high-level one
        # 'from e' preserves the original exception
        raise DataProcessingError(
            f"Failed to parse config from {filename}"
        ) from e

# The caller sees the high-level exception
try:
    config = parse_config_file("missing.conf")
except DataProcessingError as e:
    print(f"Error: {e}")
    print(f"Original cause: {e.__cause__}")

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **`raise` sin argumentos** → re-lanza excepción actual (preserva stack trace)
2. **`raise NewError() from e`** → encadena excepciones (preserva causa original)
3. **Transforma por nivel** → bajo nivel (`FileNotFoundError`) → alto nivel (`ConfigError`)
4. **Logging antes de re-lanzar** → registra contexto antes de propagar
5. **`__cause__` accesible** → código que captura puede ver excepción original

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Necesito hacer algo con este error pero también propagarlo?"
  - SÍ → captura, logea, re-lanza con `raise` ✅
  - NO → deja que se propague sin capturar ✅

- **Pregúntate**: "¿Esta excepción expone detalles de implementación?"
  - SÍ → transforma a excepción de alto nivel con `from` ✅
  - NO → re-lanza tal cual ✅

- **Pregúntate**: "¿El caller necesita saber que falló?"
  - SÍ → re-lanza o transforma ✅
  - NO → captura y maneja completamente ✅

**Cuándo usar / NO usar**:
- ✅ **Usar re-lanzamiento cuando**:
  - Necesitas logging pero caller debe manejar el error
  - Quieres añadir contexto pero no cambiar el tipo
  - Estás en middleware/interceptor
- ✅ **Usar encadenamiento cuando**:
  - Excepción de bajo nivel no es apropiada para tu API
  - Quieres ocultar detalles de implementación
  - Necesitas mantener contexto completo para debugging
- ❌ **NO usar cuando**:
  - Puedes manejar el error completamente → no re-lances
  - Excepción original ya es apropiada → no transformes

**Referencia oficial:** [Python Tutorial - Exception Chaining](https://docs.python.org/tutorial/errors.html#exception-chaining)

## 5. Mejores Prácticas

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Código en producción tiene `except: pass` en 15 lugares. Pipeline falla silenciosamente. **Datos corruptos en producción por 3 días** 💥 Nadie se dio cuenta porque errores silenciados. Pérdida: $50,000 en decisiones basadas en datos malos.

Con mejores prácticas: Excepciones específicas, logging, fail fast. **Error detectado en 5 minutos** ✅ Fix inmediato.

**Ejemplo concreto para juniors**:

Función acepta `discount_percent`. Usuario envía `150`. Sin validación: cálculo da precio negativo. Cliente "paga" -$50. **Sistema le debe dinero** 💥

Con fail fast: Validas `0 <= discount <= 100` al inicio. `ValueError` inmediato. **Error claro antes de cálculos** ✅

**Consecuencias de NO usarlo**:
- **Errores silenciados** → `except: pass` oculta problemas críticos
- **Debugging imposible** → error ocurre lejos de la causa
- **Datos corruptos** → validación tardía permite datos malos
- **Producción inestable** → errores inesperados porque capturas todo
- **Logs inútiles** → mensajes genéricos sin contexto

### 📚 El Concepto

**Definición técnica**:

Mejores prácticas son patrones probados que previenen bugs comunes y mejoran mantenibilidad.

**Principios fundamentales**:
1. **Be Specific**: Captura excepciones específicas, no genéricas
2. **Fail Fast**: Valida inputs al inicio, no después de procesamiento
3. **Provide Context**: Mensajes de error con información útil
4. **Don't Silence**: Nunca uses `except: pass` sin muy buena razón
5. **Document**: Usa `:raises:` en docstrings

**Terminología clave**:
- **Fail Fast**: Fallar inmediatamente cuando detectas problema
- **Defensive Programming**: Validar inputs y precondiciones
- **Error Context**: Información que ayuda a entender el error
- **Silent Failure**: Error que ocurre sin notificación (antipatrón)

### Principios Clave del Manejo de Errores

In [ ]:
# 1. Be specific: Catch specific exceptions
try:
    value = int(input("Enter a number: "))
except ValueError:  # GOOD: Specific
    print("Invalid number")

# 2. Don't catch everything
# try:
#     risky_operation()
# except:  # BAD: Catches everything, even KeyboardInterrupt!
#     pass

# 3. Fail fast: Validate inputs early
def calculate_discount(price, discount_percent):
    """
    Calculate discounted price with early validation.
    
    :param price: Original price
    :type price: float
    :param discount_percent: Discount percentage (0-100)
    :type discount_percent: float
    :return: Discounted price
    :rtype: float
    :raises ValueError: If inputs are invalid
    """
    # Validate early
    if price < 0:
        raise ValueError("Price cannot be negative")
    if not 0 <= discount_percent <= 100:
        raise ValueError("Discount must be between 0 and 100")
    
    return price * (1 - discount_percent / 100)

# 4. Provide context in error messages
class InvalidConfigError(Exception):
    """
    Exception with detailed context.
    """
    def __init__(self, field, value, reason):
        self.field = field
        self.value = value
        self.reason = reason
        super().__init__(
            f"Invalid config field '{field}' with value '{value}': {reason}"
        )

print("Best practices demonstrated!")

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **Específico > Genérico** → `except ValueError` no `except:`
2. **Fail Fast** → valida al inicio, no después de procesamiento
3. **Contexto en mensajes** → "Price -5 is invalid" no "Invalid input"
4. **Documenta excepciones** → `:raises ValueError:` en docstrings
5. **Nunca silencies** → `except: pass` es casi siempre malo

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Qué errores ESPECÍFICOS pueden ocurrir?"
  - Sabes → captura esos específicamente ✅
  - No sabes → NO uses `except:` genérico ❌

- **Pregúntate**: "¿Puedo validar esto ANTES de usarlo?"
  - SÍ → valida al inicio (fail fast) ✅
  - NO → maneja error cuando ocurra ✅

- **Pregúntate**: "¿Este mensaje ayuda a resolver el problema?"
  - SÍ → buen mensaje ✅
  - NO → añade más contexto ❌

**Cuándo usar / NO usar**:
- ✅ **Usar estas prácticas**:
  - Captura excepciones específicas siempre
  - Valida inputs al inicio de funciones públicas
  - Incluye valores problemáticos en mensajes
  - Documenta qué excepciones lanza tu función
  - Logea antes de re-lanzar o silenciar
- ❌ **Evita estos antipatrones**:
  - `except:` sin tipo específico
  - `except: pass` (silenciar sin logging)
  - Mensajes genéricos sin contexto
  - Validación tardía (después de procesamiento)
  - Capturar `KeyboardInterrupt` o `SystemExit`

**Patrón de validación recomendado**:
```python
def process_data(value: float, threshold: float) -> float:
    # 1. Validate inputs FIRST (fail fast)
    if value < 0:
        raise ValueError(f"Value must be positive, got {value}")
    if not 0 <= threshold <= 1:
        raise ValueError(f"Threshold must be 0-1, got {threshold}")
    
    # 2. Process (inputs are valid)
    return value * threshold
```

**Referencia oficial:** [Python Exception Handling Best Practices](https://docs.python.org/3/tutorial/errors.html#handling-exceptions)

## Ejercicios Prácticos

### 🏋️ Ejercicio 1: Validación de Email

**Objetivo**: Practicar creación de excepciones personalizadas con contexto útil

**Contexto real**: 
Sistema de registro de usuarios. Necesitas validar emails y dar mensajes claros cuando fallan.

**Instrucciones**:
1. Crea excepción `InvalidEmailError` que incluya el email inválido y la razón
2. Implementa `validate_email(email)` que valide:
   - No está vacío
   - Contiene '@'
   - Contiene '.' después del '@'
3. Lanza `InvalidEmailError` con mensaje descriptivo para cada caso
4. Retorna email normalizado (lowercase, sin espacios) si es válido

**Criterios de éxito**:
- [ ] `InvalidEmailError` tiene atributos `email` y `reason`
- [ ] Mensaje de error incluye el email y por qué es inválido
- [ ] Valida los 3 casos mencionados
- [ ] Retorna email normalizado si es válido

In [ ]:
# TODO: Implement InvalidEmailError and validate_email

# Test your implementation
# try:
#     print(validate_email("user@example.com"))  # Should work
#     print(validate_email("invalid-email"))  # Should raise InvalidEmailError
# except InvalidEmailError as e:
#     print(f"Caught: {e}")
#     print(f"Email: {e.email}, Reason: {e.reason}")

<details>
<summary><b>💡 Pista 1</b></summary>

La excepción personalizada debe heredar de `Exception` y guardar el email y la razón como atributos en `__init__`.

</details>

<details>
<summary><b>💡 Pista 2</b></summary>

Valida en orden: primero si está vacío, luego si tiene '@', luego si tiene '.' después del '@'. Usa `email.index('@')` para encontrar la posición del '@'.

</details>

<details>
<summary><b>✅ Ver Solución Completa</b></summary>

```python
class InvalidEmailError(Exception):
    """
    Raised when email validation fails.
    
    :param email: The invalid email
    :type email: str
    :param reason: Why the email is invalid
    :type reason: str
    """
    def __init__(self, email: str, reason: str):
        self.email = email
        self.reason = reason
        super().__init__(f"Invalid email '{email}': {reason}")


def validate_email(email: str) -> str:
    """
    Validate and normalize email address.
    
    :param email: Email to validate
    :type email: str
    :return: Normalized email (lowercase, trimmed)
    :rtype: str
    :raises InvalidEmailError: If email is invalid
    """
    # Fail fast: validate inputs first
    if not email or not email.strip():
        raise InvalidEmailError(email, "email cannot be empty")
    
    email = email.strip()
    
    if '@' not in email:
        raise InvalidEmailError(email, "email must contain '@'")
    
    # Check for '.' after '@'
    at_index = email.index('@')
    if '.' not in email[at_index:]:
        raise InvalidEmailError(email, "email must contain '.' after '@'")
    
    return email.lower()
```

**Explicación paso a paso**:
1. **Excepción personalizada**: Hereda de `Exception`, guarda `email` y `reason` como atributos
2. **Mensaje descriptivo**: Usa f-string para incluir email y razón en el mensaje
3. **Fail fast**: Valida vacío primero (caso más simple)
4. **Validaciones específicas**: Cada validación lanza error con razón clara
5. **Normalización**: Retorna email en lowercase si todas las validaciones pasan

**Conceptos clave aplicados**:
- **Custom exceptions**: `InvalidEmailError` con contexto útil
- **Fail fast**: Validación al inicio antes de procesamiento
- **Descriptive messages**: Cada error explica QUÉ está mal
- **Documented exceptions**: Docstring indica `:raises InvalidEmailError:`

</details>

### 🏋️ Ejercicio 2: Lectura Segura de Archivos JSON

**Objetivo**: Practicar uso de `try/except/else/finally` para manejo robusto de recursos

**Contexto real**: 
Pipeline de ML carga configuración desde archivo JSON. Archivo puede no existir, estar corrupto, o tener permisos incorrectos.

**Instrucciones**:
1. Implementa `read_json_file(filename)` que:
   - Intente abrir y parsear archivo JSON
   - Maneje `FileNotFoundError` (archivo no existe)
   - Maneje `json.JSONDecodeError` (JSON inválido)
   - Maneje `PermissionError` (sin permisos)
2. Use `else` para imprimir mensaje de éxito
3. Use `finally` para imprimir que terminó el intento
4. Retorne el contenido parseado o `None` si hay error

**Criterios de éxito**:
- [ ] Maneja las 3 excepciones específicas
- [ ] Usa `else` para código que solo corre si no hay error
- [ ] Usa `finally` para código que siempre corre
- [ ] Retorna `dict` si éxito, `None` si error

In [ ]:
import json

# TODO: Implement read_json_file

# Test your implementation
# result = read_json_file("config.json")
# print(f"Result: {result}")

<details>
<summary><b>💡 Pista 1</b></summary>

Usa `with open(filename) as f:` dentro del bloque `try`. El `with` se encarga de cerrar el archivo automáticamente.

</details>

<details>
<summary><b>💡 Pista 2</b></summary>

Estructura: `try` (abrir y parsear), `except` (3 tipos específicos), `else` (mensaje de éxito), `finally` (mensaje de finalización).

</details>

<details>
<summary><b>✅ Ver Solución Completa</b></summary>

```python
import json

def read_json_file(filename: str) -> dict | None:
    """
    Read and parse JSON file with comprehensive error handling.
    
    :param filename: Path to JSON file
    :type filename: str
    :return: Parsed JSON content or None if error
    :rtype: dict | None
    """
    try:
        with open(filename, 'r') as f:
            content = json.load(f)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON in '{filename}': {e}")
        return None
    except PermissionError:
        print(f"Error: No permission to read '{filename}'")
        return None
    else:
        # Only executes if NO exception occurred
        print(f"Successfully loaded {len(content)} items from '{filename}'")
        return content
    finally:
        # ALWAYS executes
        print(f"Finished attempt to read '{filename}'")
```

**Explicación paso a paso**:
1. **`try` block**: Intenta abrir y parsear JSON (operaciones riesgosas)
2. **`with` statement**: Context manager cierra archivo automáticamente
3. **Specific exceptions**: Captura 3 tipos específicos, no genérico
4. **`else` block**: Solo corre si NO hubo excepción (éxito)
5. **`finally` block**: SIEMPRE corre (logging de finalización)

**Conceptos clave aplicados**:
- **Specific exceptions**: Captura `FileNotFoundError`, `JSONDecodeError`, `PermissionError`
- **else vs after**: `else` hace intención más clara
- **finally for cleanup**: Garantiza que logging siempre ocurre
- **Context manager**: `with` es mejor que `try/finally` manual

**Alternativa con context manager personalizado**:
```python
# Mejor aún: usar context manager
from contextlib import contextmanager

@contextmanager
def log_operation(operation: str):
    print(f"Starting {operation}")
    try:
        yield
    finally:
        print(f"Finished {operation}")

with log_operation("reading config"):
    config = read_json_file("config.json")
```

</details>

### 🏋️ Ejercicio 3: Sistema de Transacciones Bancarias

**Objetivo**: Practicar excepciones personalizadas, propagación, y manejo completo de errores

**Contexto real**: 
API bancaria procesa transferencias. Necesitas validar saldos, montos, y manejar errores de manera clara para usuarios y logs.

**Instrucciones**:
1. Crea `InsufficientFundsError` con atributos `balance`, `amount`, `shortfall`
2. Crea `InvalidTransactionError` para montos ≤ 0
3. Implementa `transfer_money(accounts, from_account, to_account, amount)` que:
   - Valide que monto > 0 (fail fast)
   - Valide que cuentas existan
   - Valide saldo suficiente
   - Realice transferencia si todo OK
4. Implementa `process_transaction()` que capture y maneje errores apropiadamente

**Criterios de éxito**:
- [ ] Dos excepciones personalizadas con contexto útil
- [ ] Validación fail-fast al inicio
- [ ] Mensajes de error descriptivos
- [ ] Manejo apropiado en `process_transaction`

In [ ]:
# TODO: Implement the banking transaction system

# Test your implementation
# accounts = {"Alice": 1000.0, "Bob": 500.0}
# try:
#     transfer_money(accounts, "Alice", "Bob", 1500.0)
# except InsufficientFundsError as e:
#     print(f"Transaction failed: {e}")
#     print(f"Shortfall: ${e.shortfall:.2f}")

<details>
<summary><b>💡 Pista 1</b></summary>

Estructura de `InsufficientFundsError`: guarda `balance`, `amount`, calcula `shortfall = amount - balance` en `__init__`.

</details>

<details>
<summary><b>💡 Pista 2</b></summary>

Orden de validación en `transfer_money`: 1) monto > 0, 2) cuentas existen, 3) saldo suficiente, 4) transferencia.

</details>

<details>
<summary><b>✅ Ver Solución Completa</b></summary>

```python
class InsufficientFundsError(Exception):
    """
    Raised when account has insufficient funds for transaction.
    
    :param balance: Current account balance
    :type balance: float
    :param amount: Requested transaction amount
    :type amount: float
    """
    def __init__(self, balance: float, amount: float):
        self.balance = balance
        self.amount = amount
        self.shortfall = amount - balance
        super().__init__(
            f"Insufficient funds: balance=${balance:.2f}, "
            f"requested=${amount:.2f}, shortfall=${self.shortfall:.2f}"
        )


class InvalidTransactionError(Exception):
    """
    Raised when transaction amount is invalid.
    """
    pass


def transfer_money(
    accounts: dict[str, float],
    from_account: str,
    to_account: str,
    amount: float
) -> None:
    """
    Transfer money between accounts.
    
    :param accounts: Dictionary of account names to balances
    :type accounts: dict[str, float]
    :param from_account: Source account name
    :type from_account: str
    :param to_account: Destination account name
    :type to_account: str
    :param amount: Amount to transfer
    :type amount: float
    :raises InvalidTransactionError: If amount is invalid
    :raises InsufficientFundsError: If source account has insufficient funds
    :raises KeyError: If account doesn't exist
    """
    # Fail fast: validate inputs first
    if amount <= 0:
        raise InvalidTransactionError(
            f"Transaction amount must be positive, got {amount}"
        )
    
    # Check accounts exist
    if from_account not in accounts:
        raise KeyError(f"Account '{from_account}' not found")
    if to_account not in accounts:
        raise KeyError(f"Account '{to_account}' not found")
    
    # Check sufficient funds
    balance = accounts[from_account]
    if amount > balance:
        raise InsufficientFundsError(balance, amount)
    
    # Perform transfer
    accounts[from_account] -= amount
    accounts[to_account] += amount
    print(
        f"✅ Transferred ${amount:.2f} from {from_account} to {to_account}"
    )


def process_transaction(
    accounts: dict[str, float],
    from_account: str,
    to_account: str,
    amount: float
) -> bool:
    """
    Process transaction with comprehensive error handling.
    
    :param accounts: Dictionary of account names to balances
    :type accounts: dict[str, float]
    :param from_account: Source account name
    :type from_account: str
    :param to_account: Destination account name
    :type to_account: str
    :param amount: Amount to transfer
    :type amount: float
    :return: True if successful, False otherwise
    :rtype: bool
    """
    try:
        transfer_money(accounts, from_account, to_account, amount)
        return True
    except InvalidTransactionError as e:
        print(f"❌ Invalid transaction: {e}")
        return False
    except InsufficientFundsError as e:
        print(f"❌ Transaction failed: {e}")
        print(f"   You need ${e.shortfall:.2f} more")
        return False
    except KeyError as e:
        print(f"❌ Account error: {e}")
        return False


# Example usage
accounts = {"Alice": 1000.0, "Bob": 500.0}

print("Test 1: Valid transaction")
process_transaction(accounts, "Alice", "Bob", 200.0)
print(f"Balances: {accounts}\n")

print("Test 2: Insufficient funds")
process_transaction(accounts, "Bob", "Alice", 1000.0)
print(f"Balances: {accounts}\n")

print("Test 3: Invalid amount")
process_transaction(accounts, "Alice", "Bob", -50.0)
print(f"Balances: {accounts}")
```

**Explicación paso a paso**:
1. **Custom exceptions**: Dos excepciones específicas del dominio bancario
2. **Rich context**: `InsufficientFundsError` incluye balance, amount, shortfall
3. **Fail fast**: Validaciones al inicio antes de modificar estado
4. **Specific handling**: `process_transaction` maneja cada error apropiadamente
5. **User-friendly messages**: Mensajes claros con emojis y contexto

**Conceptos clave aplicados**:
- **Domain-specific exceptions**: Representan conceptos del negocio
- **Fail fast**: Valida todo antes de modificar estado
- **Error context**: Atributos con información útil para debugging
- **Separation of concerns**: `transfer_money` lanza, `process_transaction` maneja
- **Documented exceptions**: Docstrings indican qué puede lanzarse

**Mejoras posibles**:
```python
# Jerarquía de excepciones
class BankingError(Exception):
    """Base exception for banking operations."""
    pass

class InsufficientFundsError(BankingError):
    """Insufficient funds for transaction."""
    pass

class InvalidTransactionError(BankingError):
    """Invalid transaction parameters."""
    pass

# Ahora puedes capturar todas con:
except BankingError as e:
    # Maneja cualquier error bancario
    pass
```

</details>

## Resumen

En este notebook has aprendido:

1. **Try/Except básico**: Capturar excepciones específicas para manejar errores de manera controlada
2. **Else y Finally**: Usar `else` para código que solo se ejecuta si no hay errores, y `finally` para limpieza que siempre debe ocurrir
3. **Excepciones personalizadas**: Crear excepciones que representen errores específicos de tu dominio, heredando de `Exception`
4. **Propagación de errores**: Re-lanzar excepciones con `raise` y encadenarlas con `from` para preservar contexto
5. **Mejores prácticas**: Ser específico, fallar rápido, proporcionar contexto, y documentar excepciones

El manejo apropiado de errores es lo que separa código amateur de código profesional. Te permite construir aplicaciones robustas que fallan de manera predecible y proporcionan información útil para debugging.

## Preguntas de Autoevaluación

### 1. ¿Cuál es la diferencia entre el bloque `else` y simplemente poner código después del `try/except`?

**Respuesta:** El bloque `else` solo se ejecuta si NO ocurrió ninguna excepción en el bloque `try`. El código después del `try/except` se ejecuta siempre, independientemente de si hubo una excepción o no. Usar `else` hace que la intención del código sea más clara.

### 2. ¿Por qué es mejor capturar excepciones específicas en lugar de usar `except:` genérico?

**Respuesta:** Capturar excepciones específicas evita ocultar errores inesperados. Un `except:` genérico captura TODO, incluyendo `KeyboardInterrupt` y `SystemExit`, lo que puede hacer que tu programa sea imposible de detener. Además, excepciones específicas te permiten manejar diferentes errores de manera diferente.

### 3. ¿Cuándo deberías crear una excepción personalizada en lugar de usar una built-in?

**Respuesta:** Crea excepciones personalizadas cuando necesites representar errores específicos de tu dominio que no están bien representados por las excepciones built-in. Por ejemplo, `InsufficientFundsError` es más descriptivo que `ValueError` en un sistema bancario. Las excepciones personalizadas también pueden incluir atributos adicionales con contexto útil.

### 4. ¿Qué hace `raise` sin argumentos dentro de un bloque `except`?

**Respuesta:** `raise` sin argumentos re-lanza la excepción actual que está siendo manejada. Esto es útil cuando quieres hacer algo (como logging) pero también quieres que el código que llamó tu función pueda manejar la excepción.

### 5. ¿Cuál es el propósito de `raise NewException() from original_exception`?

**Respuesta:** Esta sintaxis encadena excepciones, preservando la excepción original como causa de la nueva. Esto es útil cuando transformas excepciones de bajo nivel en excepciones de alto nivel más apropiadas para tu API, pero quieres mantener el contexto completo para debugging. La excepción original queda accesible en el atributo `__cause__`.

### 6. ¿Por qué es importante validar inputs al inicio de una función ("fail fast")?

**Respuesta:** Validar inputs temprano evita que datos inválidos se propaguen a través de tu código, causando errores confusos más adelante. Es más fácil debuggear cuando el error ocurre cerca de donde se introdujo el problema. Además, proporciona mensajes de error más claros al usuario.

### 7. ¿Qué información debería incluir una buena excepción personalizada?

**Respuesta:** Una buena excepción personalizada debe incluir: (1) Un nombre descriptivo que termine en `Error`, (2) Un mensaje claro explicando qué salió mal, (3) Atributos con contexto relevante (como valores que causaron el error), (4) Documentación en el docstring explicando cuándo se lanza.

## Recursos y Referencias Oficiales

### Documentación Oficial
- **[Python Tutorial - Errors and Exceptions](https://docs.python.org/tutorial/errors.html)**: Tutorial oficial sobre manejo de errores y excepciones en Python
- **[Built-in Exceptions](https://docs.python.org/3/library/exceptions.html)**: Referencia completa de todas las excepciones built-in de Python y su jerarquía

### Estándares/PEPs
- **[PEP 352 - Required Superclass for Exceptions](https://www.python.org/dev/peps/pep-0352/)**: Define la jerarquía de excepciones y por qué todas deben heredar de `BaseException`

### Mejores Prácticas
- **[Real Python - Python Exceptions](https://realpython.com/python-exceptions/)**: Guía completa sobre excepciones con ejemplos prácticos
- **[Python Exception Handling Best Practices](https://docs.python-guide.org/writing/gotchas/#mutable-default-arguments)**: Patrones y anti-patrones comunes en manejo de excepciones

### Notas Importantes
- Todos los enlaces están actualizados a partir de 2025
- Se recomienda revisar la documentación oficial regularmente
- La jerarquía de excepciones puede variar ligeramente entre versiones de Python